In [168]:
using UrlDownload, Plots, DataFrames, Random,BenchmarkTools, BetaML
import DecisionTree, Flux


url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/heart/heart.dat";
feature_names = ["age", "sex", "chest_pain", "rest_bp", "serum_cholestrol", "fasting_blood_sugar", "rest_ECG", "max_HR", "ex_ind_ang", "oldpeak",
"slope", "major_vessel", "thal","heart_dis"]
data = urldownload(url, true, format = :CSV, header = feature_names);
s_log = DataFrame(data)
describe(s_log)


,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,age,54.4333,29.0,55.0,77.0,0,Float64
2,sex,0.677778,0.0,1.0,1.0,0,Float64
3,chest_pain,3.17407,1.0,3.0,4.0,0,Float64
4,rest_bp,131.344,94.0,130.0,200.0,0,Float64
5,serum_cholestrol,249.659,126.0,245.0,564.0,0,Float64
6,fasting_blood_sugar,0.148148,0.0,0.0,1.0,0,Float64
7,rest_ECG,1.02222,0.0,2.0,2.0,0,Float64
8,max_HR,149.678,71.0,153.5,202.0,0,Float64
9,ex_ind_ang,0.32963,0.0,0.0,1.0,0,Float64


In [169]:
#import Pkg; Pkg.add("BenchmarkTools")

In [170]:
x     = Matrix{Union{Missing,Float64}}(s_log[:,1:13]);
y     = Vector{Int64}(s_log[:,14]);

In [171]:
xFull = predictMissing(x,rng=copy(FIXEDRNG)).X̂;


Iter. 1:	Var. of the post  13.914203140512832 	  Log-likelihood -7709.961707028344


In [172]:
y_oh  = oneHotEncoder(y);

In [173]:
((xtrain,xtest),(xtrainFull,xtestFull),(ytrain,ytest),(ytrain_oh,ytest_oh)) = partition([x,xFull,y,y_oh],[0.8,1-0.8],rng=copy(FIXEDRNG));

In [174]:
myForest = buildForest(xtrain,ytrain,30, rng=copy(FIXEDRNG),forceClassification=true);

In [175]:
ŷtrain,ŷtest   = predict.(Ref(myForest), [xtrain,xtest],rng=copy(FIXEDRNG));

In [176]:
trainAccuracy,testAccuracy  = accuracy.([parse.(Int64,mode(ŷtrain,rng=copy(FIXEDRNG))),parse.(Int64,mode(ŷtest,rng=copy(FIXEDRNG)))],[ytrain,ytest])

2-element Vector{Float64}:
 0.9953703703703703
 0.7777777777777778

In [119]:
cm = ConfusionMatrix(parse.(Int64,mode(ŷtest,rng=copy(FIXEDRNG))),ytest,classes=[1,2],labels=["Absent","Present"])
print(cm;what="all")

3×3 Matrix{Any}:
 "Labels"     "Absent"    "Present"
 "Absent"   27           6
 "Present"   6          15

3×3 Matrix{Any}:
 "Labels"    "Absent"   "Present"
 "Absent"   0.818182   0.181818
 "Present"  0.285714   0.714286


-----------------------------------------------------------------

*** CONFUSION MATRIX ***

Scores actual (rows) vs predicted (columns):


Normalised scores actual (rows) vs predicted (columns):


 *** CONFUSION REPORT ***

- Accuracy:               0.7777777777777778
- Misclassification rate: 0.2222222222222222
- Number of classes:      2

  N Class   precision   recall  specificity  f1Score  actualCount  predictedCount
                          TPR       TNR                 support                  

  1 Absent      0.818    0.818        0.714    0.818           33              33
  2 Present     0.714    0.714        0.818    0.714           21              21

- Simple   avg.    0.766    0.766        0.766    0.766
- Weigthed avg.    0.778    0.778        0.755    0.778

-----------------------------------------------------------------


In [120]:
####Now compare it with Decision Tree

In [121]:
#train the model
model = DecisionTree.build_forest(ytrain, xtrainFull,-1,30,rng=123)
# generate predictions and measure their error
(ŷtrain,ŷtest) = DecisionTree.apply_forest.([model],[xtrainFull,xtestFull]);
(trainAccuracy,testAccuracy) = accuracy.([ŷtrain,ŷtest],[ytrain,ytest])

2-element Vector{Float64}:
 0.9861111111111112
 0.7777777777777778

In [122]:
########## Neural Network ############

In [123]:
xScaleFactors   = getScaleFactors(xtrainFull)
D               = size(xtrainFull,2)
classes         = unique(y)
nCl             = length(classes)

2

In [124]:
ls   = 50
l1   = DenseLayer(D,ls,f=relu,rng=copy(FIXEDRNG))
l2   = DenseLayer(ls,nCl,f=relu,rng=copy(FIXEDRNG))

DenseLayer([-0.21670047185742425 0.11485258740575172 … 0.015708445469304444 -0.143712203658498; -0.09002285443322663 -0.3106534072484348 … 0.24140674578878807 -0.0467179750052083], [0.03062628460300243, 0.09404664993620965], BetaML.Utils.relu, nothing)

In [125]:
l3   = VectorFunctionLayer(nCl,f=softmax) ## Add a (parameterless) layer whose activation function (softMax in this case) is defined to all its nodes at once

VectorFunctionLayer{0}(fill(NaN), 2, 2, BetaML.Utils.softmax, nothing, nothing)

In [126]:
mynn = buildNetwork([l1,l2,l3],squaredCost,name="Multinomial logistic regression Model heart disease") ## Build the NN and use the MSE as error function (crossEntropy could also be used)

NN(AbstractLayer[DenseLayer([-0.19687530959080626 0.11544496297917989 … -0.05598223094485649 0.002138517600364709; -0.08178698082600586 -0.09013840862941555 … -0.2700364115819889 0.004476485381907591; … ; -0.025915512537422913 -0.13056448075414953 … -0.026913579804940246 0.26042043933950687; 0.20742643302401387 -0.04244391216027166 … -0.07352930707299968 -0.30143554079789964], [0.2475958451621727, 0.08226907268205336, 0.10388889417149283, -0.11131697180073483, -0.1468940776225006, -0.027258225608090658, -0.07019643251425495, -0.2886476021065192, -0.039210506664476674, -0.06713446918227134  …  -0.1439254733419904, -0.1853363218137804, -0.20352858780757255, 0.0015028967970229234, -0.05504984654735395, -0.2176419341768248, -0.1463224541567973, -0.1449790503871383, 0.15786425167021867, -0.2829014303108609], BetaML.Utils.relu, nothing), DenseLayer([-0.21670047185742425 0.11485258740575172 … 0.015708445469304444 -0.143712203658498; -0.09002285443322663 -0.3106534072484348 … 0.241406745788788

In [127]:
res  = train!(mynn,scale(xtrainFull,xScaleFactors),ytrain_oh,epochs=500,batchSize=8,optAlg=ADAM(),rng=copy(FIXEDRNG)) ## Use optAlg=SGD() to use Stochastic Gradient Descent instead

***
*** Training Multinomial logistic regression Model heart disease for 500 epochs with algorithm ADAM.
Training.. 	 avg ϵ on (Epoch 1 Batch 27): 	 0.21225197383788996
Training.. 	 avg ϵ on (Epoch 50 Batch 27): 	 0.008174163355146803

Training the Neural Network... 15%|████                 |  ETA: 0:00:06


Training.. 	 avg ϵ on (Epoch 100 Batch 27): 	 0.03298905809207948

Training the Neural Network... 31%|███████              |  ETA: 0:00:05


Training.. 	 avg ϵ on (Epoch 150 Batch 27): 	 0.0025191231062001306
Training.. 	 avg ϵ on (Epoch 200 Batch 27): 	 0.12687079321287373

...


Training.. 	 avg ϵ on (Epoch 400 Batch 27): 	 3.803606874906938e-5
Training.. 	 avg ϵ on (Epoch 450 Batch 27): 	 7.46933296213425e-5

Training the Neural Network... 93%|████████████████████ |  ETA: 0:00:00

Training the Neural Network...100%|█████████████████████| Time: 0:00:06


Training.. 	 avg ϵ on (Epoch 500 Batch 27): 	 0.03133596506003456
Training of 500 epoch completed. Final epoch error: 0.013984188596164645.


(epochs = 500, ϵ_epochs = [0.279893136638733, 0.21751017208249013, 0.18153477654884326, 0.1553164186812575, 0.13017651674106367, 0.11364356265222048, 0.0995538698425807, 0.09153132179960696, 0.08523512282494052, 0.08149005277437722  …  0.013912064543859482, 0.013911285553495982, 0.013910566298040843, 0.013909895049314391, 0.013909192223368309, 0.013908590711507351, 0.013908049017475362, 0.013907544395619515, 0.013952913721953477, 0.013984188596164645], θ_epochs = Any[])

In [128]:
(ŷtrain,ŷtest)  = predict.(Ref(mynn),[scale(xtrainFull,xScaleFactors),scale(xtestFull,xScaleFactors)])
trainAccuracy   = accuracy(ŷtrain,ytrain,rng=copy(FIXEDRNG))

0.9861111111111112

In [129]:
testAccuracy    = accuracy(ŷtest,ytest,rng=copy(FIXEDRNG))

0.7777777777777778

In [130]:
cm = ConfusionMatrix(ŷtest,ytest,classes=[1,2],labels=["Absent","Present"],rng=copy(FIXEDRNG))
print(cm)

3×3 Matrix{Any}:
 "Labels"     "Absent"    "Present"
 "Absent"   26           7
 "Present"   5          16

3×3 Matrix{Any}:
 "Labels"    "Absent"   "Present"
 "Absent"   0.787879   0.212121
 "Present"  0.238095   0.761905


-----------------------------------------------------------------

*** CONFUSION MATRIX ***

Scores actual (rows) vs predicted (columns):


Normalised scores actual (rows) vs predicted (columns):


 *** CONFUSION REPORT ***

- Accuracy:               0.7777777777777778
- Misclassification rate: 0.2222222222222222
- Number of classes:      2

  N Class   precision   recall  specificity  f1Score  actualCount  predictedCount
                          TPR       TNR                 support                  

  1 Absent      0.839    0.788        0.762    0.812           33              31
  2 Present     0.696    0.762        0.788    0.727           21              23

- Simple   avg.    0.767    0.775        0.775    0.770
- Weigthed avg.    0.783    0.778        0.772    0.779

-----------------------------------------------------------------


In [131]:
##################        FLUX         #######################################################

In [132]:
xtrainT, ytrain_ohT = transpose.([scale(xtrainFull,xScaleFactors), ytrain_oh])
xtestT, ytest_ohT   = transpose.([scale(xtestFull,xScaleFactors), ytest_oh])    #since Flux requires the data in fields and observations

2-element Vector{LinearAlgebra.Transpose{Float64, Matrix{Float64}}}:
 [-0.6189858705759482 -0.5092149772718392 … 1.0275775289856883 -0.9482985504882756; 0.7144854865492444 -1.3996085558430407 … 0.7144854865492444 0.7144854865492444; … ; 2.4851402543173067 -0.7163923198330208 … 0.35078520488375503 -0.7163923198330208; 1.1928142960683614 -0.8724994349999038 … 1.1928142960683614 1.1928142960683614]
 [0.0 1.0 … 1.0 1.0; 1.0 0.0 … 0.0 0.0]

In [133]:
Random.seed!(123)

l1         = Flux.Dense(D,ls,Flux.relu)
l2         = Flux.Dense(ls,ls,Flux.relu)
l3         = Flux.Dense(ls,nCl,Flux.relu)
Flux_nn    = Flux.Chain(l1,l3)
loss(x, y) = Flux.logitcrossentropy(Flux_nn(x), y)
ps         = Flux.params(Flux_nn)
nndata     = Flux.Data.DataLoader((xtrainT, ytrain_ohT), batchsize=8,shuffle=true)
begin for i in 1:500  Flux.train!(loss, ps, nndata, Flux.ADAM()) end end
ŷtrain     = Flux.onecold(Flux_nn(xtrainT),1:3)
ŷtest      = Flux.onecold(Flux_nn(xtestT),1:3)
trainAccuracy =  accuracy(ŷtrain,ytrain)

1.0

In [134]:
testAccuracy  = accuracy(ŷtest,ytest)

0.7407407407407407

In [135]:
###################### Processing times ##########################

In [136]:
@btime buildForest(xtrain,ytrain,30, rng=copy(FIXEDRNG),forceClassification=true);

  144.009 ms (458357 allocations: 109.86 MiB)


In [137]:
@btime  DecisionTree.build_forest(ytrain, xtrainFull,-1,30,rng=123);

  2.582 ms (8491 allocations: 1.32 MiB)


In [138]:
@btime train!(mynn,scale(xtrainFull),ytrain_oh,epochs=500,batchSize=8,rng=copy(FIXEDRNG),verbosity=NONE);

  5.813 s (40431782 allocations: 4.74 GiB)


In [139]:
@btime begin for i in 1:500 Flux.train!(loss, ps, nndata, Flux.ADAM()) end end;

  928.258 ms (4021000 allocations: 713.82 MiB)
